In [1]:
!curl -X GET "https://datasets-server.huggingface.co/first-rows?dataset=SPRINGLab%2FIndicTTS_Telugu&config=default&split=train"


{"dataset":"SPRINGLab/IndicTTS_Telugu","config":"default","split":"train","features":[{"feature_idx":0,"name":"audio","type":{"_type":"Audio"}},{"feature_idx":1,"name":"text","type":{"dtype":"string","_type":"Value"}},{"feature_idx":2,"name":"gender","type":{"names":["female","male"],"_type":"ClassLabel"}}],"rows":[{"row_idx":0,"row":{"audio":[{"src":"https://datasets-server.huggingface.co/assets/SPRINGLab/IndicTTS_Telugu/--/7ab3c18491bef3f5d4e820c2701b5f78f54031d6/--/default/train/0/audio/audio.wav?Expires=1762943080&Signature=kyNIEkDY61rCaL6Y7GCQZCbN~aHb~cSHbxn7MrDGXv-dvoBJM8eMsOFDSh03nvhET2aVzps~fQcDG2fwOa1tN7fGureDrOGDTWJ9R2umZ9c2zXV2KKSdRRJaS3vzacep1x02SQMSoY4AyU4i7s9lO5U7UKg2qAfhUAr9daN~kGUBlkcJwa7Y8JGFwQvbbnynfXNAOqFDX7LT1C4TZw6SxcI081sb9jFiC6~9kiFXJmE9x3MZMFH4k-A6TpHuff~wt1jdbgQAwSJYsQfl3cyPyjA-UpqqFFBczGCEM8Bov5X2YYPykeJpc75B~lVKPwW1vYzZjZ1zzB5yrWbXlDMcxg__&Key-Pair-Id=K3EI6M078Z3AC3","type":"audio/wav"}],"text":"నరకవాసి.","gender":0},"truncated_cells":[]},{"row_idx":1,"row":{

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100  117k  100  117k    0     0  54401      0  0:00:02  0:00:02 --:--:-- 54485


In [2]:
!curl -X GET "https://datasets-server.huggingface.co/splits?dataset=SPRINGLab%2FIndicTTS_Telugu"


{"splits":[{"dataset":"SPRINGLab/IndicTTS_Telugu","config":"default","split":"train"}],"pending":[],"failed":[]}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   112  100   112    0     0    133      0 --:--:-- --:--:-- --:--:--   133


In [3]:
# Step 1: Install dependencies (run this once)
!pip install datasets soundfile

# Step 2: Import libraries
from datasets import load_dataset
import re

# Step 3: Load the dataset from Hugging Face
dataset = load_dataset("SPRINGLab/IndicTTS_Telugu", split="train")

print("Total samples:", len(dataset))
print("Sample row:", dataset[0])


Defaulting to user installation because normal site-packages is not writeable


C:\Users\MIKKY\AppData\Roaming\Python\Python38\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Total samples: 8576
Sample row: {'audio': {'path': 'train_telugufemale_00001.wav', 'array': array([0.00082397, 0.00079346, 0.00082397, ..., 0.        , 0.        ,
       0.        ]), 'sampling_rate': 16000}, 'text': 'నరకవాసి.', 'gender': 0}


In [4]:
import os
import soundfile as sf
from tqdm import tqdm

# Make folder for saving WAVs
os.makedirs("wavs", exist_ok=True)

for i, example in enumerate(tqdm(dataset)):
    audio = example["audio"]
    array = audio["array"]
    sr = audio["sampling_rate"]

    # Save as .wav file
    out_path = f"wavs/{i:06d}.wav"
    sf.write(out_path, array, sr)


100%|██████████| 8576/8576 [02:56<00:00, 48.52it/s] 


In [5]:
pip install "huggingface_hub[hf_xet]"

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ------- -------------------------------- 0.5/2.9 MB 2.4 MB/s eta 0:00:01
   ------------------ --------------------- 1.3/2.9 MB 2.5 MB/s eta 0:00:01
   ------------------------- -------------- 1.8/2.9 MB 2.5 MB/s eta 0:00:01
   -------------------------------- ------- 2.4/2.9 MB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 2.9/2.9 MB 2.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
with open("metadata.csv", "w", encoding="utf-8") as f:
    for i, example in enumerate(dataset):
        wav = f"wavs/{i:06d}.wav"
        text = example["text"].replace("|", " ")  # avoid delimiter conflict
        f.write(f"{wav}|{text}\n")


In [10]:
import torch
print(torch.__file__)


C:\Users\MIKKY\AppData\Roaming\Python\Python38\site-packages\torch\__init__.py


In [8]:
lexicon = {}
with open("telugu_lexicon.txt", "r", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) > 1:
            lexicon[parts[0]] = " ".join(parts[1:])

train_lines = []
with open("metadata.csv", "r", encoding="utf-8") as f:
    for line in f:
        wav, text = line.strip().split("|")
        phonemes = " ".join(lexicon.get(word, word) for word in text.split())
        train_lines.append(f"{wav}|{phonemes}|0\n")

# 90/10 split
n = int(0.9 * len(train_lines))
os.makedirs("filelists", exist_ok=True)
with open("filelists/train.txt", "w", encoding="utf-8") as f:
    f.writelines(train_lines[:n])
with open("filelists/val.txt", "w", encoding="utf-8") as f:
    f.writelines(train_lines[n:])


In [12]:
# Check what the 'gender' field actually looks like
print(dataset.features["gender"])
print("Example gender values from dataset:")
print([dataset[i]["gender"] for i in range(10)])


ClassLabel(names=['female', 'male'], id=None)
Example gender values from dataset:
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
# Filter only female samples
female_samples = dataset.filter(lambda x: x["gender"] == 0)

print("Total female samples:", len(female_samples))


Filter: 100%|██████████| 8576/8576 [01:14<00:00, 115.32 examples/s]

Total female samples: 4043


In [21]:
telugu_texts = [sample["text"] for sample in female_samples]
print("Total text entries:", len(telugu_texts))
print("Sample text:", telugu_texts[0])


Total text entries: 4043
Sample text: నరకవాసి.


In [22]:
all_words = []
for text in telugu_texts:
    words = text.strip().split()
    all_words.extend(words)


In [23]:
unique_words = sorted(set(all_words))
print("Total unique words:", len(unique_words))


Total unique words: 36784


In [24]:
with open("telugu_unique_words.txt", "w", encoding="utf-8") as f:
    for word in unique_words:
        f.write(word + "\n")

print("✅ Unique words saved to telugu_unique_words.txt")


✅ Unique words saved to telugu_unique_words.txt
